Trained on Kaggle environment.  
Try to train locally with conda environment, succesfully instal but the kernel is always get error and stop (always need restart). It happens to while training on Google Collaboratory environtment, so i trained/modelling on Kaggle.

In [3]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=829224 sha256=4ab70e186bdcd259d0f1e9e6e53d5aed8e4dec2ff1df1439a00125c1eac051a5
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


In [4]:
import pandas as pd
import numpy as np

from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k

import pickle

# Load Data

In [5]:
interaction_df = pd.read_csv('/kaggle/input/data-wisata/interaction_data.csv')
user_df = pd.read_csv('/kaggle/input/data-wisata/user_data.csv')
wisata_df = pd.read_csv('/kaggle/input/data-wisata/wisata_data.csv')
interaction_df.head(), user_df.head(), wisata_df.head()

(  user_id wisata_id  rating
 0      U1        W3       4
 1      U1        W1       4
 2      U1        W6       4
 3      U1        W4       5
 4      U1        W7       1,
   user_id  umur asal_kota jenis_kelamin
 0      U1    55    Kediri             P
 1      U2    45    Blitar             P
 2      U3    31    Jember             P
 3      U4    24  Pasuruan             P
 4      U5    37    Kediri             P,
   wisata_id             nama_wisata kategori   harga  \
 0        W1            Gunung Bromo     Alam   35000   
 1        W2              Kawah Ijen     Alam  100000   
 2        W3           Pantai Klayar     Alam   10000   
 3        W4         Telaga Sarangan     Alam   15000   
 4        W5  Air Terjun Tumpak Sewu     Alam   10000   
 
                                        fasilitas       lokasi  
 0  Parkir, Toilet, Warung, Spot Foto, Penginapan  Probolinggo  
 1       Parkir, Toilet, Warung, Spot Foto, Guide   Banyuwangi  
 2              Parkir, Toilet, Warung,

# Preprocessing

## Normaliasi Rating

In [6]:
# rubah rating menjadi implicit feedback (bobot)
interaction_df['rating'] = interaction_df['rating'] / 5.0
interaction_df.head()

,user_id,wisata_id,rating
0,U1,W3,0.8
1,U1,W1,0.8
2,U1,W6,0.8
3,U1,W4,1.0
4,U1,W7,0.2


## Feature Engineering (User)

In [7]:
# rubah 'user' menjadi token fitur
user_df['feature'] = (
    "umur_" + user_df['umur'].astype(str) + " " +
    "kota_" + user_df['asal_kota'] + " " + 
    "gender_" + user_df['jenis_kelamin'] 
)

user_features = dict(zip(user_df['user_id'], user_df['feature']))

## Feature Engineering (Wisata)

In [8]:
wisata_df['feature'] = (
    "kategori_" + wisata_df['kategori'] + " " +
    "lokasi_" + wisata_df['lokasi'] + " " +
    wisata_df['fasilitas'].str.replace(",", "")
)

item_features = dict(zip(wisata_df['wisata_id'], wisata_df['feature']))

# Dataset

In [9]:
dataset = Dataset()

dataset.fit(
    users = interaction_df['user_id'],
    items = interaction_df['wisata_id'],
    user_features = set(" ".join(user_features.values()).split()),
    item_features = set(" ".join(item_features.values()).split())
)

## Interaction Matrix

In [10]:
# buat matriks interaksi dari dataset
(interactions, weights) = dataset.build_interactions(
    [(row['user_id'], row['wisata_id'], row['rating'])
     for _, row in interaction_df.iterrows()]
)

## Feature Matrix

In [11]:
# user - features
user_features_matrix = dataset.build_user_features(
    [(u, f.split()) for u, f in user_features.items()]
)

In [12]:
# item - features
item_features_matrix = dataset.build_item_features(
    [(i, f.split()) for i, f in item_features.items()]
)

# Training Model

In [13]:
# inisialisasi model
model = LightFM(
    loss = 'warp',
    no_components = 10,
    learning_rate = 0.05,
    random_state = 42
)

In [14]:
# train
model.fit(
    interactions,
    user_features = user_features_matrix,
    item_features = item_features_matrix,
    epochs = 8,
    num_threads = 1
)

# Evaluasi

In [15]:
precision = precision_at_k(
    model,
    interactions,
    user_features=user_features_matrix,
    item_features=item_features_matrix,
    k=5
).mean()

recall = recall_at_k(
    model,
    interactions,
    user_features=user_features_matrix,
    item_features=item_features_matrix,
    k=5
).mean()

print(f"Precision@5: {precision:.4f}")
print(f"Recall@5: {recall:.4f}")


Precision@5: 0.5786
Recall@5: 0.6750


# Save Atrifact

## Model

In [16]:
with open("lightfm_model.pkl", "wb") as f:
    pickle.dump(model, f)

## Dataset Mapping

In [17]:
# Extract mappings dari dataset object
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

print(f"✓ User ID Map: {len(user_id_map)} users")
print(f"✓ User Feature Map: {len(user_feature_map)} features")
print(f"✓ Item ID Map: {len(item_id_map)} items")
print(f"✓ Item Feature Map: {len(item_feature_map)} features")

✓ User ID Map: 28 users
✓ User Feature Map: 60 features
✓ Item ID Map: 8 items
✓ Item Feature Map: 24 features


In [18]:
# Save user_id_map
with open("user_id_map.pkl", "wb") as f:
    pickle.dump(user_id_map, f)
print("✓ User ID mapping saved")

# Save item_id_map
with open("item_id_map.pkl", "wb") as f:
    pickle.dump(item_id_map, f)
print("✓ Item ID mapping saved")

# Save user_feature_map
with open("user_feature_map.pkl", "wb") as f:
    pickle.dump(user_feature_map, f)
print("✓ User feature mapping saved")

# Save item_feature_map
with open("item_feature_map.pkl", "wb") as f:
    pickle.dump(item_feature_map, f)
print("✓ Item feature mapping saved")

✓ User ID mapping saved
✓ Item ID mapping saved
✓ User feature mapping saved
✓ Item feature mapping saved


## Save Mappings (Terpisah)

## Feature Matrix

In [19]:
with open("user_features.pkl", "wb") as f:
    pickle.dump(user_features_matrix, f)

with open("item_features.pkl", "wb") as f:
    pickle.dump(item_features_matrix, f)

## Data Masterr

In [20]:
user_df.to_csv("user_master.csv", index=False)
wisata_df.to_csv("wisata_data.csv", index=False)  # Renamed untuk backend
print("✓ Master data saved")

✓ Master data saved


In [21]:
print("\n" + "="*60)
print("🎉 All artifacts saved successfully!")
print("="*60)
print("\nFiles created:")
print("  📦 lightfm_model.pkl")
print("  📦 user_id_map.pkl")
print("  📦 item_id_map.pkl")
print("  📦 user_feature_map.pkl")
print("  📦 item_feature_map.pkl")
print("  📦 user_features.pkl")
print("  📦 item_features.pkl")
print("  📄 user_master.csv")
print("  📄 wisata_data.csv")
print("\n📝 Next steps:")
print("  1. Copy semua file .pkl ke folder '../models/'")
print("  2. Copy wisata_data.csv ke folder '../models/'")
print("  3. Restart backend server")
print("="*60)


🎉 All artifacts saved successfully!

Files created:
  📦 lightfm_model.pkl
  📦 user_id_map.pkl
  📦 item_id_map.pkl
  📦 user_feature_map.pkl
  📦 item_feature_map.pkl
  📦 user_features.pkl
  📦 item_features.pkl
  📄 user_master.csv
  📄 wisata_data.csv

📝 Next steps:
  1. Copy semua file .pkl ke folder '../models/'
  2. Copy wisata_data.csv ke folder '../models/'
  3. Restart backend server
